# Project plots
Containing programs for plotting.

In [53]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
import numpy as np

def load_pareto_info_dict():
    ''' Load the Pareto solutions information dictionary. '''
    pareto_info_dict = pd.read_pickle('pareto_info_dict.pkl')
    path_and_attack_properties_df = pareto_info_dict['path_and_attack_properties_df']
    pareto_optimals = pareto_info_dict['pareto_optimals']
    pareto_fronts = pareto_info_dict['pareto_fronts']
    
    return path_and_attack_properties_df, pareto_optimals, pareto_fronts

In [54]:
def interpolate_color(start_color, end_color, steps):
    ''' Custom color gradient function. '''
    start_rgb = np.array(to_rgb(start_color))
    end_rgb = np.array(to_rgb(end_color))
    colors = [(start_rgb + (end_rgb - start_rgb) * i / (steps - 1)).tolist() for i in range(steps)]
    
    return colors

def generate_gradient_colors():
    ''' Generate 10 gradient colors of green, yellow and red. '''
    green_col = '#4ca648'
    yellow_col = '#d9a300'
    red_col = '#c04b4b'
    gradient_colors = interpolate_color(green_col, yellow_col, 5) + interpolate_color(yellow_col, red_col, 5)
    
    return gradient_colors

def plot_multiple_pareto_fronts(df=path_and_attack_properties_df, pareto_fronts=pareto_fronts, criteria1='Success coefficient', criteria2='Expected troop number'):
    ''' Visualize multiple Pareto fronts with gradient colors. '''
    sns.set_theme(style='darkgrid')
    plt.figure(figsize=(10, 7))
    gradient_colors = generate_gradient_colors()
    
    # Plot all metrics in scatter plot. 
    sns.scatterplot(x=criteria1, y=criteria2, data=df,
                    color='gray', alpha=0.3, label='All Points',)
    
    # Color Pareto front points.
    for i, pareto_front in enumerate(pareto_fronts):
        sns.scatterplot(x=criteria1, y=criteria2, data=pareto_front,
                        color=gradient_colors[i], label=f'Pareto Front {i+1}', alpha=0.8,)
        
        # Sort and plot the line for the Pareto front.
        pareto_front_sorted = pareto_front.sort_values(by=criteria1)
        plt.plot(pareto_front_sorted[criteria1], pareto_front_sorted[criteria2],
                 color=gradient_colors[i], linestyle="--", alpha=0.8,)
    
    # Set labels and title.
    plt.xticks(rotation=45)
    plt.xlabel(criteria1, fontsize=12)
    plt.ylabel(criteria2, fontsize=12)
    plt.title('First 10 Pareto fronts of solutions for conquering the graph', fontsize=14)
    
    # Move the legend to the right of the plot.
    plt.legend(bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)
    plt.tight_layout()
    plt.show()

In [55]:
def generate_pareto_optimal_bar_plot():
    ''' Generates a bar plot with the first Pareto front and its metrics. '''
    # Combine node path and choice of nodes to attack from.
    pareto_optimals['combined'] = pareto_optimals.apply(
        lambda row: f"{row['Node path']}\n{row['Attacking nodes']}", axis=1)
    pareto_optimals['combined']
    
    # Create the figure and axis objects.
    fig, ax1 = plt.subplots(figsize=(14, 8))

    # Plot the red bars (Expected troop number) on the left y-axis.
    ax1.bar(pareto_optimals['combined'], pareto_optimals['Expected troop number'], 
            color='#c04b4b', label='Expected troop number', alpha=0.8, width=0.4, align='center')
    ax1.set_ylabel('Expected troop number', color='black', fontsize=14, labelpad=10)
    ax1.tick_params(axis='y', labelcolor='black')

    # Create a second y-axis for the green bars (Success coefficient).
    ax2 = ax1.twinx()
    ax2.bar(pareto_optimals['combined'], pareto_optimals['Success coefficient'], 
            color='#4ca648', label='Success coefficient', alpha=0.8, width=0.4, align='edge')
    ax2.set_ylabel('Success coefficient', color='black', fontsize=14, labelpad=15)
    ax2.tick_params(axis='y', labelcolor='black')

    # Set xticks.
    ax1.set_xticks(range(len(pareto_optimals['combined'])))
    ax1.set_xticklabels(pareto_optimals['combined'], rotation=45, ha='right', fontsize=10)

    # Customize the plot title.
    ax1.set_xlabel('Options for order of territories to attack and which territories to attack from', fontsize=18)
    plt.title('Optimal decisions for conquering the graph', fontsize=20)

    # Add legends for both y-axes.
    fig.legend(loc='upper right', bbox_to_anchor=(0.4, 0.5), fontsize=14, title='Metric', title_fontsize=14)

    # Ensure layout is adjusted for the legend and rotated x-tick labels.
    plt.tight_layout()

    # Show the plot.
    plt.show()

In [56]:
path_and_attack_properties_df, pareto_optimals, pareto_fronts = load_pareto_info_dict()